In [1]:
import requests
import json
import bs4
import time
import numpy as np
import pandas as pd

import sys
import signal

from itertools import islice   


import CONFIG

In [2]:
def getBSFromURL(url):
    try:
        r = requests.get(url, headers = CONFIG.HEADER)
        return getBS(r.text)
    
    except requests.exceptions.RequestException as e:
        print("Connection Error")

def getBS(html):
    html_bs = bs4.BeautifulSoup(html, 'html.parser')
    return html_bs

In [3]:
# a very simple sleeping function to avoid getting banned
def ImNotRobot():
    sleepTime = np.random.uniform(5,10)
    print(f"sleeping {sleepTime} seconds")
    time.sleep(sleepTime)
    
class BlockedError(Exception):
    pass

def getAbstract(url):
    metadata_id = url.split('/')[-1]
    
    r = requests.get('https://www.jstor.org/stable/content-metadata/'+metadata_id, headers = CONFIG.HEADER)
    
    ## check if blocked
    if not r: 
        raise BlockedError("You might be blocked")
    
    article_data = json.loads(r.text)

    abstract = article_data['abstract']
    if len(abstract) == 0 : return ''
    
    return abstract[0]

def save():
    articles_df.to_csv(file_name, index = False)
    
def saveAndQuit(sig=None, frame=None):
    save()
    sys.exit(0)


In [9]:
file_name = f'{CONFIG.JOURNAL_NAME}.csv'

articles_df = pd.read_csv(file_name)
not_yet_scraped_articles_df = articles_df[(articles_df['scraped'] == 0) & (articles_df['Month'] != 'May')]
not_yet_scraped_articles_df

,Source URL,Title,Long Volume,Issue Vol.,Issue No.,Month,Year,Abstract,Author 1,Author 2,Author 3,Author 4,Author 5,Author 6,Author 7,Author 8,scraped
2199,https://www.jstor.org/stable/1806085,The Evans and Heckman Subadditivity Test: Comment,"Vol. 76, No. 4, Sep., 1986",Vol. 76,No. 4,Sep.,1986,NaN,Toshiyuki Sueyoshi,Peter C. Anselmo,NaN,NaN,NaN,NaN,NaN,NaN,0
2200,https://www.jstor.org/stable/1806086,A Test For Subadditivity of the Cost Function ...,"Vol. 76, No. 4, Sep., 1986",Vol. 76,No. 4,Sep.,1986,NaN,David S. Evans,James J. Heckman,NaN,NaN,NaN,NaN,NaN,NaN,0
2201,https://www.jstor.org/stable/1806087,New Estimates of the Value of Federal Mineral ...,"Vol. 76, No. 4, Sep., 1986",Vol. 76,No. 4,Sep.,1986,NaN,Michael J. Boskin,Marc S. Robinson,Terrance O'Reilly,Praveen Kumar,NaN,NaN,NaN,NaN,0
2202,https://www.jstor.org/stable/1813352,"Life Cycle, Individual Thrift, and the Wealth ...","Vol. 76, No. 3, Jun., 1986",Vol. 76,No. 3,Jun.,1986,NaN,Franco Modigliani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2203,https://www.jstor.org/stable/1813353,Is the Stabilization of the Postwar Economy a ...,"Vol. 76, No. 3, Jun., 1986",Vol. 76,No. 3,Jun.,1986,NaN,Christina D. Romer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3581,https://www.jstor.org/stable/1813671,Minimizing AEA Convention Transportation Costs,"Vol. 69, No. 5, Dec., 1979",Vol. 69,No. 5,Dec.,1979,NaN,John J. Siegfried,Larry Nelson,NaN,NaN,NaN,NaN,NaN,NaN,0
3582,https://www.jstor.org/stable/1813672,Hedging and the Competitive Firm Under Price U...,"Vol. 69, No. 5, Dec., 1979",Vol. 69,No. 5,Dec.,1979,NaN,Duncan M. Holthausen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3583,https://www.jstor.org/stable/1813673,Unanticipated Money Growth and Unemployment in...,"Vol. 69, No. 5, Dec., 1979",Vol. 69,No. 5,Dec.,1979,NaN,David H. Small,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3584,https://www.jstor.org/stable/1813674,Unanticipated Money Growth and Unemployment in...,"Vol. 69, No. 5, Dec., 1979",Vol. 69,No. 5,Dec.,1979,NaN,Robert J. Barro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [10]:
signal.signal(signal.SIGINT, saveAndQuit)

counter = 1
for index, articles in not_yet_scraped_articles_df.iterrows():
    ImNotRobot()
    try:
        url = articles['Source URL']
        abstract = getAbstract(url)
        print (abstract)
        articles_df.loc[index, 'Abstract'] = abstract
        articles_df.loc[index, 'scraped'] = 1
        
        if counter == 10 :
            save()
            counter = 0
        counter += 1
    except Exception as e:
        print (e)
        saveAndQuit()
        
    

sleeping 6.3250884476442275 seconds

sleeping 6.326765963670982 seconds

sleeping 5.334286761342469 seconds

sleeping 9.221232760266934 seconds

sleeping 9.943327550316408 seconds

sleeping 9.334549310078067 seconds
This paper presents estimates of the distributional welfare impacts of the actual price rises of energy and nonenergy commodities during the 1970-80 decade in the United States. Measures of welfare change based on net compensating variations are computed for different types of families. These show that welfare differences due to changes in commodity prices are minor compared to welfare differences due to different income growth patterns over the decade.
sleeping 8.38211894524714 seconds

sleeping 7.703389678439564 seconds

sleeping 8.870171476230091 seconds

sleeping 5.0820870054419505 seconds

sleeping 7.455302112158426 seconds
This paper analyzes data on union and employer rankings of different panels of arbitrators in an actual arbitration system. A random utility model 


sleeping 8.90273545788175 seconds

sleeping 9.396152523603746 seconds

sleeping 9.616448582029257 seconds

sleeping 7.4292870612187425 seconds

sleeping 5.329020050333348 seconds

sleeping 9.572539310018565 seconds

sleeping 6.696484004420065 seconds

sleeping 5.314984524013312 seconds

sleeping 9.704320111004474 seconds

sleeping 8.984721706173023 seconds

sleeping 9.011780566302997 seconds

sleeping 9.743895917944176 seconds

sleeping 8.06422904315593 seconds

sleeping 8.717293909119793 seconds

sleeping 8.856228617064144 seconds

sleeping 7.372880550834472 seconds

sleeping 6.735559026274506 seconds

sleeping 9.628819334626861 seconds


SystemExit: 0

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
r = requests.get('https://www.jstor.org/stable/content-metadata/1813673', headers = CONFIG.HEADER).text

In [14]:
rj=json.loads(r)

In [15]:
rj['abstract']

[]

In [48]:
getAbstract('https://www.jstor.org/stable/116861')

BlockedError: You might be blocked

In [20]:
articles_df = pd.read_csv(file_name)

In [21]:
articles_df

,Source URL,Title,Long Volume,Issue Vol.,Issue No.,Month,Year,Abstract,Author 1,Author 2,Author 3,Author 4,Author 5,Author 6,Author 7,Author 8,scraped
0,https://www.jstor.org/stable/116860,The Value of Weather Information Services for ...,"Vol. 88, No. 5, Dec., 1998",Vol. 88,No. 5,Dec.,1998,The U.S. government established a national wea...,Erik D. Craft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,https://www.jstor.org/stable/116861,The Rise and Fall of Bank Control in the Unite...,"Vol. 88, No. 5, Dec., 1998",Vol. 88,No. 5,Dec.,1998,This article studies how equity ownership and ...,Miguel Cantillo Simon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,https://www.jstor.org/stable/116862,Winners and Losers in Russia's Economic Transi...,"Vol. 88, No. 5, Dec., 1998",Vol. 88,No. 5,Dec.,1998,The transition to a market economy has produce...,Elizabeth Brainerd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,https://www.jstor.org/stable/116863,Unemployment and the Social Safety Net during ...,"Vol. 88, No. 5, Dec., 1998",Vol. 88,No. 5,Dec.,1998,We investigate the remarkably short unemployme...,John C. Ham,Jan Svejnar,Katherine Terrell,NaN,NaN,NaN,NaN,NaN,1
4,https://www.jstor.org/stable/116864,Federalism and the Soft Budget Constraint,"Vol. 88, No. 5, Dec., 1998",Vol. 88,No. 5,Dec.,1998,The government's incentives to bail out ineffi...,Yingyi Qian,Gérard Rol,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3581,https://www.jstor.org/stable/1813671,Minimizing AEA Convention Transportation Costs,"Vol. 69, No. 5, Dec., 1979",Vol. 69,No. 5,Dec.,1979,NaN,John J. Siegfried,Larry Nelson,NaN,NaN,NaN,NaN,NaN,NaN,0
3582,https://www.jstor.org/stable/1813672,Hedging and the Competitive Firm Under Price U...,"Vol. 69, No. 5, Dec., 1979",Vol. 69,No. 5,Dec.,1979,NaN,Duncan M. Holthausen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3583,https://www.jstor.org/stable/1813673,Unanticipated Money Growth and Unemployment in...,"Vol. 69, No. 5, Dec., 1979",Vol. 69,No. 5,Dec.,1979,NaN,David H. Small,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3584,https://www.jstor.org/stable/1813674,Unanticipated Money Growth and Unemployment in...,"Vol. 69, No. 5, Dec., 1979",Vol. 69,No. 5,Dec.,1979,NaN,Robert J. Barro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
